In [58]:
%load_ext autoreload
%autoreload 2

from descriptor import *
from dataset import *
from template import *
from matplotlib import pyplot as plt
import os
import cv2
import pickle

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [60]:
curr_wd = os.getcwd()
annot_source_path = os.path.join(curr_wd, 'cache_anno', 'classification')
annot_source_path_provided = os.path.join(annot_source_path, 'provided_annots.pkl')
annot_source_path_extra = os.path.join(annot_source_path, 'extra_annots.pkl')

extract_provided_annots()
extract_extra_annots()

with open(annot_source_path_provided, 'rb') as f:
    provided_annots = pickle.load(f)

with open(annot_source_path_extra, 'rb') as f:
    extra_annots = pickle.load(f)

In [61]:
extra_annots

{'000.jpg': [[[739, 4924, 861, 5086], ['wenda', 'face', 'front']],
  [[7940, 6288, 8050, 6438], ['other', 'face', 'front']],
  [[7932, 6057, 8050, 6207], ['other', 'face', 'front']],
  [[9710, 6109, 9816, 6271], ['other', 'face', 'front']],
  [[8017, 4291, 8102, 4457], ['other', 'face', 'front']],
  [[9588, 3954, 9689, 4149], ['waldo', 'face', 'front']],
  [[7506, 861, 7607, 995], ['other', 'face', 'front']],
  [[9048, 5549, 9166, 5728], ['other', 'face', 'front']],
  [[6663, 6626, 6796, 6795], ['other', 'face', 'front']],
  [[9109, 2484, 9210, 2642], ['other', 'face', 'front']],
  [[9764, 1666, 9861, 1820], ['other', 'face', 'front']],
  [[3514, 3743, 3619, 3889], ['other', 'face', 'front']],
  [[1997, 4604, 2143, 4755], ['other', 'face', 'front']],
  [[977, 5139, 1112, 5301], ['other', 'face', 'front']],
  [[7481, 2332, 7611, 2483], ['other', 'face', 'front']],
  [[8647, 5597, 8798, 5770], ['other', 'face', 'front']],
  [[3287, 5144, 3400, 5263], ['other', 'face', 'front']]],
 '019.j

In [64]:
# create positive
positive = {}
class_name = 'wenda'
img_src_path = os.path.join(curr_wd, 'datasets', 'JPEGImages')

for k, v in provided_annots.items():
    source_name = f'JPEGImages/{k}'
    if source_name not in positive:
        positive[source_name] = []
        
    for b in v:
        coordinate = b[0]
        labels = b[1]
        if labels[0] == class_name:
            x = coordinate[0]
            y = coordinate[1]
            w = coordinate[2] - x
            h = coordinate[3] - y
            positive[source_name].append([x, y, w, h])
            
for k, v in extra_annots.items():
    source_name = f'JPEGImages/{k}'
    if source_name not in positive:
        positive[source_name] = []
        
    for b in v:
        coordinate = b[0]
        labels = b[1]
        if labels[0] == class_name:
            x = coordinate[0]
            y = coordinate[1]
            w = coordinate[2] - x
            h = coordinate[3] - y
            positive[source_name].append([x, y, w, h])

In [65]:
positive

{'JPEGImages/016.jpg': [[8253, 1856, 149, 258], [8235, 1827, 134, 155]],
 'JPEGImages/002.jpg': [],
 'JPEGImages/003.jpg': [[912, 804, 20, 32], [909, 798, 23, 40]],
 'JPEGImages/017.jpg': [[4343, 4303, 85, 144],
  [259, 4018, 185, 440],
  [282, 4013, 109, 109],
  [4333, 4292, 101, 98],
  [258, 3999, 182, 455],
  [271, 4005, 154, 194]],
 'JPEGImages/001.jpg': [],
 'JPEGImages/015.jpg': [[1814, 4572, 167, 162]],
 'JPEGImages/029.jpg': [[1029, 4287, 135, 291],
  [1019, 4277, 146, 201],
  [1014, 4294, 135, 161],
  [1029, 4283, 146, 216],
  [1010, 4280, 150, 194]],
 'JPEGImages/028.jpg': [],
 'JPEGImages/014.jpg': [[1818, 4430, 98, 144],
  [1814, 4408, 108, 104],
  [1812, 4412, 108, 159]],
 'JPEGImages/000.jpg': [[734, 4935, 122, 204], [739, 4924, 122, 162]],
 'JPEGImages/038.jpg': [[5518, 2519, 130, 369], [5505, 2520, 160, 176]],
 'JPEGImages/004.jpg': [[2230, 645, 708, 1954],
  [2346, 606, 335, 471],
  [2244, 610, 483, 2025]],
 'JPEGImages/010.jpg': [],
 'JPEGImages/011.jpg': [[4567, 3815

In [66]:
pos_info_path = os.path.join(curr_wd, 'datasets', 'info.dat')
with open(pos_info_path, 'w') as fp:
    for k, v in positive.items():
        if len(v) > 0:
            fp.write(f'{k} {len(v)}')
            for b in v:
                fp.write(f' {b[0]} {b[1]} {b[2]} {b[3]}')
            fp.write('\n')

In [32]:
# create negative
from dataset import *
img_lst = []
img_lst_pth = os.path.join(curr_wd, 'datasets', 'ImageSets', 'images.txt')
with open(img_lst_pth, 'r') as fp:
    contents = fp.readlines()
    for l in contents:
        file_name = l.strip()
        if file_name:
            img_lst.append(file_name)

save_random_images(img_lst, 8)            

In [35]:
negative_path = os.path.join(curr_wd, 'datasets', 'bg')
dirs = os.listdir(negative_path)
neg_info_path = os.path.join(curr_wd, 'datasets', 'bg.txt')
with open(neg_info_path, 'w') as fp:
    for dir in dirs:
        fp.write(f'bg/{dir}\n')